In [38]:
import json
import os
from typing import Any, Callable, Optional, Dict, List, Tuple

directory = "C:\Coding\PycharmProjects\EquityRiseAndFallTearSheet\Data"
file_name = 'Service_Token.json'

file_path = os.path.join(directory, file_name)

dict_token = {'Last Update': None, 'Current Update': None}

In [39]:
import datetime as dt
import pytz

my_tz = pytz.timezone('Europe/Berlin')
utc_now = my_tz.normalize(my_tz.localize(dt.datetime.now(), is_dst=True))

#encoded_time_stamp_utc =

now = dt.datetime.now()

In [51]:
import json
import datetime as dt
from datetime import timezone
from datetime import datetime
from datetime import timedelta
import pytz
from dateutil import parser

class DateTimeEncoder(json.JSONEncoder):

    def default(self, obj):
        if isinstance(obj, (dt.datetime, dt.date, dt.time)):
            return obj.isoformat()
        elif isinstance(obj, dt.timedelta):
            return (dt.datetime.min + obj).time().isoformat()

        return super(DateTimeEncoder, self).default(obj)

class UTCDateTimeEncoder(json.JSONEncoder):

    def default(self, obj: Any) -> Any:
        if isinstance(obj, datetime):
            return {
                '_type' : 'datetime',
                'year' : obj.year,
                'month' : obj.month,
                'day' : obj.day,
                'hour' : obj.hour,
                'minute' : obj.minute,
                'second' : obj.second,
                'microsecond' : obj.microsecond,
                'int_rep': self._encode(obj, datetime(1970, 1, 1, tzinfo=timezone.utc))
            }
        else:
            return super(DateTimeEncoder, self).default(obj)

    @staticmethod
    def _encode(input_dt, epoch):
        return (input_dt - epoch) / timedelta(seconds=1)

class UTCDateTimeDecoder(json.JSONDecoder):


    def __init__(self, *args, **kargs) -> None:
        json.JSONDecoder.__init__(self, object_hook= self.dict_to_object
                                  ,*args ,**kargs)
        self.my_tz = pytz.timezone('UTC')

    def dict_to_object(self, obj):
        if '_type' not in obj:
            return obj
        type = obj['_type']

        if type == 'datetime':
            time_stamp = obj['int_rep']
            encoded_time_stamp=dt.datetime.utcfromtimestamp(time_stamp)
            return self.my_tz.normalize(self.my_tz.localize(encoded_time_stamp, is_dst=True))
        return obj

        try:
            return {idx_key: dt.utcfromtimestamp(time_stamp)}
            #return
        except:
            return input



if os.path.exists(file_path):
    with open(file_path) as file:
        dict_token_3 = json.loads(file.read(), cls=UTCDateTimeDecoder)

dict_token['Last Update'] = dict_token['Current Update']
dict_token['Current Update'] = utc_now

try:
    with open(file_path, "w") as file:
        serialized_dic = json.dumps(dict_token, cls=UTCDateTimeEncoder, indent=2)
        file.write(serialized_dic)
except FileNotFoundError as err:
    print(err)

In [40]:
dt.datetime.u